# Python Debugging with __`pdb`__
* __`pdb`__ can be run
  * from the commandline
  * from within the Python interpreter
  * from within your Python program
  * from within Jupyter
* important to distinguish between using __`pdb`__ to run or step through a program, vs. using __`pdb`__ in post-mortem mode, to determine why a program crashed
  * we will demonstrate both...

## Let's first run from within Jupyter...
* In order to do that, it's helpful to be familiar with a couple of Jupyter's "magic" commands
   * __`%debug`__ = start __`pdb`__ after a crash
   * __`%pdb`__ = toggle automatic calling of __`pdb`__ after a crash
   * __`%run`__ = run a named file inside of Jupyter
     * -d = run your program under the control of __`pdb`__
     * -t = output approximate timing information
     * -N__n__ (used with -t) = run the program __n__ times

## First we'll create a crash and do a post-mortem debug...
* Here's some code which computes prime numbers using the Sieve of Eratosthenes
* Using __`random()`__, I've hard-coded a bug which occurs 0.5% of the time, causing a crash


In [ ]:
%pdb

In [ ]:
%debug

In [ ]:
from random import random
limit = 100
numbers = list(range(2, limit + 1))
primes = []

while numbers:
    candidate = numbers[0]
    primes.append(candidate)
    for num in range(candidate, limit + 1, candidate):
        if num in numbers:
            if random() > 0.99:
                raise ValueError
            else:
                numbers.remove(num)

print(primes)

### Basic __`pdb`__ commands
 * __s(tep)__ = step, i.e., move one line ahead–stops inside a called function
 * __n(ext)__ = next, i.e., move one line ahead–executes called functions at (nearly) full speed, only stopping at the next line in the current function
 * __l(ist)__ = list code
   * current line in the current frame is indicated by `->`
   * if an exception is being debugged, the line where the exception was originally raised or propagated is indicated by `>>`, if it differs from the current line
 * __b(reak) `func`__ = breakpoint inside function `func`
 * __b(reak) `num`__ = breakpoint at line number `num`
 * __t(break)__ = same as __b__, but breakpoint is removed after first hit
 * __cl(ear)__ = clear breakpoints
 * __unt(il) `num`__ = continue execution until a line number >= `num` is reached
 * __r(eturn)__ = continue execution until current function returns
 * __j(ump) `num`__ = jump back to line `n` (can't jump into a loop)
 * __p `expression`__ = print `expression`
 * __pp `expression`__ = pretty print `expression`
 * __q(uit)__ = quit

### Next, let's run an "outside" script directly from within Jupyter
* and then let's try running it with debugging turned on

In [ ]:
%run primes.py # !python3 primes.py

In [ ]:
# might have to restart the kernel before doing this
%run -d primes.py

### To run from the command line, we need to jump out of the notebook for a moment...
* The command we'll use is

    __`python3 -m pdb script.py`__

### Now that we're back, we can run from the interpreter...
* This will allow us to test a module without having to do the save/run/import cycle

In [ ]:
class PdbTester:
    def __init__(self, count):
        self.count = count
        
    def doit(self):
        for i in range(self.count):
            print(i)

In [ ]:
p = PdbTester(5)
p.doit()

In [ ]:
from mypdb import PdbTester # importing PdbTester from mypdb.py
import pdb
pdb.run('PdbTester(5).doit()')

### What if we have a program where the bug doesn't occur until the program has been running for a while?
* In a case such as this, we're going to want to invoke __`pdb`__ from *within* our program...
* ...but not from within the notebook, so we'll once again drop out of the notebook to examine the following code:

In [ ]:
for num in range(1, 1000): # do something
    pass

print('do a lot more stuff')
x = 1
print('and now we get to the bug...')
if x == 1: # this is the buggy case
    import pdb
    pdb.set_trace() # causes the program to stop here, in the debugger
    print('buggy line')
    y = 1
else:
    y = x ** 2 # no debugging in this case
print(x)

# Lab
* consider the BankAccount class we tinkered with earlier today
* suppose there was a bug in the __`withdraw()`__ method and that was deep down in our code, not easy to get to
* instrument __`withdraw()`__ so the debugger will start running during a withdrawal so we could debug the code